In [1]:
import procedural
import json
import os

master_path = "/home/kongsgaard/git/procedural-py/"
secret_path = os.path.join(master_path, "secrets")
data_path = os.path.join(master_path, "examples", "data")


In [2]:
user, password, url = procedural.utils.get_secrets(secret_path)
client = procedural.Client(url, user, password)
print(f"CLIENT: {client}")


CLIENT: http://localhost:8001 - Authenticated: Yes


In [3]:
project_name = "Jupyter Project"
project_number = 99
project = client.get_or_create("api/project/", {"name": project_name, "number": project_number})
project_id = project.get("uid")

print(f"PROJECT: {project_id[:8]}")
print(json.dumps(project, indent=4))

PROJECT: 8e7b022e
{
    "uid": "8e7b022e-4363-408f-80c8-c5a4e4246a32",
    "name": "Jupyter Project",
    "number": 99,
    "cluster": "e93cc55f-e3cc-4020-aa2b-0711757c476f",
    "path": "/mnt/cluster/projects/99_jupyter-project",
    "path_type": "name",
    "company": "1575b6cf-41d3-4996-8dcc-73045c1d9539"
}


In [4]:
task_name = "Jupyter Wind Thresholds"
parent_task = client.get_or_create(
    f"api/project/{project.get('uid')}/task/",
    {"name": task_name},
    {"config": {
        "task_type": "parent",
        "case_dir": "foam"
    }
    })
parent_id = parent_task.get("uid")

print(f"PARENT TASK: {parent_id[:8]}")
print(json.dumps(parent_task, indent=4))


PARENT TASK: 6d4fd5f9
{
    "uid": "6d4fd5f9-6ef8-419b-a445-8898b468ed6f",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-10-12T11:15:57.847963",
    "config": {
        "task_type": "parent",
        "case_dir": "foam"
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-wind-thresholds_59fee6e1",
    "parent": null,
    "meta_json": {},
    "project": "8e7b022e-4363-408f-80c8-c5a4e4246a32",
    "name": "Jupyter Wind Thresholds",
    "dependent_on": null,
    "priority": 100
}


In [5]:
stl_path = os.path.join(data_path, "mesh_vwt", "cfdGeom.stl")
file_upload = client.update(f"api/task/{parent_id}/file/foam/constant/triSurface/cfdGeom.stl/",
                            files=open(stl_path, 'r').read(), extra_headers={"content-type": "application/octet-stream"})
print(f"UPLOADED FILE: {file_upload.get('file')}")

UPLOADED FILE: foam/constant/triSurface/cfdGeom.stl


In [6]:
write_mesh_path = os.path.join(data_path, "mesh_vwt", "write_mesh.json")
with open(write_mesh_path, 'r') as f:
    mesh_config = json.load(f)

payload = {"name": "Actions", "config": mesh_config, "parent": parent_id}
action_task = client.create(f"/api/project/{project_id}/task/", payload)
action_id = action_task.get("uid")

write_solution_path = os.path.join(data_path, "mesh_vwt", "write_solution.json")
with open(write_solution_path, 'r') as f:
    solution_config = json.load(f)

payload = {"name": "Actions", "status": "pending", "config": solution_config}
client.update(f"/api/project/{project_id}/task/{action_id}/", payload)

print(f"ACTION TASK: {action_id[:8]}")
print(json.dumps(action_task, indent=4))

configs = client.get(f"/api/task/{action_id}/config/")
print(f"TASK CONFIGS FOR ACTION TASK: {action_id[:8]}")
print(json.dumps(configs, indent=4))

ACTION TASK: 1aa8e413
{
    "uid": "1aa8e413-9504-46c0-89e0-838a4e715442",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-10-12T11:15:58.097128",
    "config": {
        "cmd": "cfd.io.tasks.write_mesh",
        "case_dir": "foam",
        "base_mesh": {
            "type": "simpleBox",
            "cell_size": 2,
            "parameters": {
                "z0": "True",
                "square": "True"
            },
            "bounding_box": {
                "max": [
                    35,
                    35,
                    24
                ],
                "min": [
                    -35,
                    -35,
                    0
                ]
            }
        },
        "task_type": "magpy",
        "snappyhex_mesh": "dict data too large for JSON."
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-wind-thresholds_59fee6e1",
    "parent": {
        "uid": "6d4fd5f9-6ef8-419b-a445-

In [7]:
cpus = [2, 1, 1]
payload = {
    "name": "Mesh",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "cpus": cpus,
        "commands": [
            "blockMesh",
            "snappyHexMesh -overwrite",
            "reconstructParMesh -constant -mergeTol 1e-6",
            "!checkMesh -writeSets vtk"
        ]},
    "parent": parent_id,
    "dependent_on": action_id
}

mesh_task = client.create(f"/api/project/{project_id}/task/", payload)
mesh_id = mesh_task.get("uid")

print(f"MESH TASK: {mesh_id[:8]}")
print(json.dumps(mesh_task, indent=4))


MESH TASK: e24c67c3
{
    "uid": "e24c67c3-3fa5-493e-b35c-f7b79c17b6b8",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-10-12T11:15:58.503020",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "cpus": [
            2,
            1,
            1
        ],
        "commands": [
            "blockMesh",
            "snappyHexMesh -overwrite",
            "reconstructParMesh -constant -mergeTol 1e-6",
            "!checkMesh -writeSets vtk"
        ],
        "case_dir": "foam"
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-wind-thresholds_59fee6e1",
    "parent": {
        "uid": "6d4fd5f9-6ef8-419b-a445-8898b468ed6f",
        "name": "Jupyter Wind Thresholds"
    },
    "meta_json": {},
    "project": "8e7b022e-4363-408f-80c8-c5a4e4246a32",
    "name": "Mesh",
    "dependent_on": {
        "uid": "1aa8e413-9504-46c0-89e0-838a4e715442",
        "name": "Actions"
    },
    "priority

In [11]:
payload_vwt = {
    "name": "VWT",
    "config": {
        "task_type": "cfd",
        "cmd": "wind_tunnel",
        "commands": [
            0,
            45,
            90,
            135,
            180,
            225,
            270,
            315,
        ],
        "cpus": cpus,
        "iterations": {
            "init": 100,
            "run": 50
        }
    },
    "parent": parent_id,
    "dependent_on": mesh_id
}

vwt_task = client.create(f"/api/project/{project_id}/task/", payload_vwt)
vwt_id = vwt_task.get("uid")

print(f"VWT TASK: {vwt_id[:8]}")
print(json.dumps(vwt_task, indent=4))

VWT TASK: 76c253f5
{
    "uid": "76c253f5-38c9-4582-bd79-ff7dbe19ca2d",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-10-12T11:20:03.986379",
    "config": {
        "task_type": "parent",
        "cmd": "wind_tunnel",
        "commands": [
            0,
            45,
            90,
            135,
            180,
            225,
            270,
            315
        ],
        "cpus": [
            2,
            1,
            1
        ],
        "iterations": {
            "init": 100,
            "run": 50
        },
        "case_dir": "foam"
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-wind-thresholds_59fee6e1",
    "parent": {
        "uid": "6d4fd5f9-6ef8-419b-a445-8898b468ed6f",
        "name": "Jupyter Wind Thresholds"
    },
    "meta_json": {},
    "project": "8e7b022e-4363-408f-80c8-c5a4e4246a32",
    "name": "VWT",
    "dependent_on": null,
    "priority": 100
}


In [12]:
probe_sets_path = os.path.join(data_path, "mesh_vwt", "probe_sets.json")
with open(probe_sets_path, 'r') as f:
    sets = json.load(f)

payload = {
    "name": "Probing",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "commands": [
            "write_sample_set",
            "postProcess -fields '(U p)' -func internalCloud"
        ],
        "case_dir": "VWT/",
        "cpus": [1, 1, 1],
        "sets": sets,
        "fields": ["Utrans", ]
    },
    "parent": parent_id,
    "dependent_on": vwt_id
}
probe_task = client.create(f"/api/project/{project_id}/task/", payload)
probe_id = probe_task.get("uid")

print(f"PROBE TASK: {probe_id[:8]}")
print(json.dumps(probe_task, indent=4))


PROBE TASK: 0540874d
{
    "uid": "0540874d-aab5-4267-97b5-9fc7f349a5c1",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-10-12T11:21:23.279603",
    "config": {
        "task_type": "cfd",
        "cmd": "pipeline",
        "commands": [
            "write_sample_set",
            "postProcess -fields '(U p)' -func internalCloud"
        ],
        "case_dir": "VWT/",
        "cpus": [
            1,
            1,
            1
        ],
        "sets": "list data too large for JSON.",
        "fields": [
            "Utrans"
        ]
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-wind-thresholds_59fee6e1",
    "parent": {
        "uid": "6d4fd5f9-6ef8-419b-a445-8898b468ed6f",
        "name": "Jupyter Wind Thresholds"
    },
    "meta_json": {},
    "project": "8e7b022e-4363-408f-80c8-c5a4e4246a32",
    "name": "Probing",
    "dependent_on": {
        "uid": "76c253f5-38c9-4582-bd79-ff7dbe19ca2d",
        "na

In [14]:
epw_name = "DNK_Copenhagen.061800_IWEC.epw"
epw_path = os.path.join(data_path, "mesh_vwt", epw_name)
file_upload = client.update(f"api/task/{parent_id}/file/WeatherFiles/{epw_name}/",
                            files=open(epw_path, 'r').read(), extra_headers={"content-type": "application/octet-stream"})
print(f"UPLOADED EPW FILE: {file_upload.get('file')}")

payload = {
    "name": "Wind Thresholds",
    "config": {
        "task_type": "cfd",
        "cmd": "run_wind_thresholds",
        "case_dir": "VWT/",
        "cpus": [2, 1, 1],
        "patches": ["set1"],
        "epw_file": f"WeatherFiles/{epw_name}",
        "set_foam_patch_fields": False
    },
    "parent": parent_id,
    "dependent_on": probe_id
}
threshold_task = client.create(f"/api/project/{project_id}/task/", payload)
threshold_id = threshold_task.get("uid")

print(f"WIND THRESHOLD TASK: {threshold_id[:8]}")
print(json.dumps(threshold_task, indent=4))


UPLOADED EPW FILE: WeatherFiles/DNK_Copenhagen.061800_IWEC.epw
WIND THRESHOLD TASK: 067d2aad
{
    "uid": "067d2aad-15dd-4762-a804-07d4f5d50ac6",
    "status": "waiting",
    "started": null,
    "stopped": null,
    "created": "2020-10-12T11:22:10.109394",
    "config": {
        "task_type": "cfd",
        "cmd": "run_wind_thresholds",
        "case_dir": "VWT/",
        "cpus": [
            2,
            1,
            1
        ],
        "patches": [
            "set1"
        ],
        "epw_file": "WeatherFiles/DNK_Copenhagen.061800_IWEC.epw",
        "set_foam_patch_fields": false
    },
    "cluster_base_dir": "/mnt/cluster/projects/99_jupyter-project/jupyter-wind-thresholds_59fee6e1",
    "parent": {
        "uid": "6d4fd5f9-6ef8-419b-a445-8898b468ed6f",
        "name": "Jupyter Wind Thresholds"
    },
    "meta_json": {},
    "project": "8e7b022e-4363-408f-80c8-c5a4e4246a32",
    "name": "Wind Thresholds",
    "dependent_on": {
        "uid": "0540874d-aab5-4267-97b5-9fc7f

In [ ]:
#client.delete(f"api/project/{project_id}/task/{parent_id}/")
#client.delete(f"api/project/{project_id}/")